In [1]:
import tsl
import torch
import numpy as np
from electric_data import ElectricData
import pandas as pd

In [2]:
data = pd.read_csv('./data/electric/normal_data.csv')
imputation_mask = np.random.rand(*data.shape) > 0.1
imputation_mask

array([[ True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True],
       ...,
       [ True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True],
       [ True, False,  True,  True,  True,  True]])

In [3]:
data = ElectricData(mask=imputation_mask)

In [4]:
data.has_mask

True

In [5]:
data.covariates

{}

In [6]:
print(data.similarity_score)

{'pearson'}


In [7]:
data.get_similarity('pearson')

array([[ 1.        ,  0.62491763, -0.65651678, -0.51319127,  0.50083874,
         0.08541757],
       [ 0.62491763,  1.        , -0.08369033,  0.07618859,  0.6663897 ,
         0.36590527],
       [-0.65651678, -0.08369033,  1.        ,  0.87355349,  0.07062312,
         0.03894102],
       [-0.51319127,  0.07618859,  0.87355349,  1.        ,  0.05986351,
        -0.15136584],
       [ 0.50083874,  0.6663897 ,  0.07062312,  0.05986351,  1.        ,
        -0.0631576 ],
       [ 0.08541757,  0.36590527,  0.03894102, -0.15136584, -0.0631576 ,
         1.        ]])

In [8]:
adj = data.get_connectivity(method='pearson')

In [9]:
target, idx = data.numpy(return_idx=True)

In [10]:
from tsl.data import ImputationDataset

In [11]:
torch_dataset = ImputationDataset(
    target=target,
    index=idx,
    connectivity=adj,
    input_mask=data.mask,
    eval_mask=data.mask,
    window=24,
    stride=1)

In [12]:
from tsl.data import SpatioTemporalDataModule
from tsl.data.preprocessing import StandardScaler, MinMaxScaler

scalers = {'target': MinMaxScaler(axis=(0,1))}

splitter = data.get_splitter(val_len=0.1, test_len=0.2)

dm = SpatioTemporalDataModule(
    dataset=torch_dataset,
    scalers=scalers,
    splitter=splitter,
    batch_size=64,
    workers=0)
print(dm)

dm.setup()
print(dm)

SpatioTemporalDataModule(train_len=None, val_len=None, test_len=None, scalers=[target], batch_size=64)
SpatioTemporalDataModule(train_len=1550, val_len=150, test_len=437, scalers=[target], batch_size=64)


In [13]:
from tsl.metrics.torch import MaskedMAE, MaskedMAPE
from tsl.engines import Imputer
from tsl.nn.models import GRINModel

loss_fn = MaskedMAE()

metrics = {'mae': MaskedMAE(),
           'mape': MaskedMAPE(),
           'mae_at_15': MaskedMAE(at=2),  # '2' indicates the third time step,
                                          # which correspond to 15 minutes ahead
           'mae_at_30': MaskedMAE(at=5),
           'mae_at_60': MaskedMAE(at=11), }

model_kwargs = {
    'input_size': dm.n_channels,
    'embedding_size':3,
    'n_nodes':dm.n_nodes,
    'hidden_size': 64,
    'ff_size': 64,
    'embedding_size': 8,
    'n_layers': 1,
    'kernel_size': 2,
    'decoder_order': 1,
    'layer_norm': False,
    'dropout': 0,
    'ff_dropout': 0,
    'merge_mode': 'mlp',
}

# setup predictor
predictor = Imputer(
    model_class=GRINModel,
    model_kwargs=model_kwargs,
    optim_class=torch.optim.Adam,
    optim_kwargs={'lr': 0.003},
    loss_fn=loss_fn,
    metrics=metrics
)

In [14]:
from pytorch_lightning.loggers import TensorBoardLogger

logger = TensorBoardLogger(save_dir="logs", name="tsl_intro", version=0)

In [15]:
import warnings
warnings.filterwarnings("ignore", ".*does not have many workers.*")